<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

## Libraries Import  

<br/><br/>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import plot_roc_curve, RocCurveDisplay, roc_auc_score
from sklearn.metrics import plot_precision_recall_curve, PrecisionRecallDisplay, average_precision_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# run to avoid warnings
import warnings
warnings.filterwarnings('ignore')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovateable:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [ ]:
# load the data
house = pd.read_csv('datasets/housing.csv')

In [ ]:
# look at the dataset
house.head()

In [ ]:
house.tail()

In [ ]:
# check the shape
house.shape

In [ ]:
# check the dataset info
house.info()

In [ ]:
# check for duplicate rows
house.duplicated().sum()

In [ ]:
# drop Id column
house.drop(columns='Id', inplace=True)

In [ ]:
# build a data dictionary which includes description, mutability and variable type for all features
data_dict = {'MSSubClass':['Identifies the type of dwelling involved in the sale', 'fixed', 'nominal'],
             'MSZoning':['Identifies the general zoning classification of the sale', 'fixed', 'nominal'],
             'LotFrontage':['Linear feet of street connected to property', 'fixed', 'continuous'],
             'LotArea':['Lot size in square feet', 'fixed', 'continuous'],
             'Street':['Type of road access to property', 'mutable', 'nominal'],
             'Alley':['Type of alley access to property', 'mutable', 'nominal'], 
             'LotShape':['General shape of property', 'fixed', 'nominal'], 
             'LandContour':['Flatness of the property', 'fixed', 'nominal'], 
             'Utilities':['Type of utilities available', 'mutable', 'nominal'], 
             'LotConfig':['Lot configuration', 'fixed', 'nominal'],
             'LandSlope':['Slope of property', 'fixed', 'nominal'], 
             'Neighborhood':['Physical locations within Ames city limits', 'fixed', 'nominal'], 
             'Condition1':['Proximity to various conditions', 'fixed', 'nominal'], 
             'Condition2':['Proximity to various conditions (if more than one is present)', 'fixed', 'nominal'], 
             'BldgType':['Type of dwelling', 'fixed', 'nominal'],
             'HouseStyle':['Style of dwelling', 'fixed', 'nominal'], 
             'OverallQual':['Rates the overall material and finish of the house', 'mutable', 'ordinal'], 
             'OverallCond':['Rates the overall condition of the house','mutable', 'ordinal'], 
             'YearBuilt':['Original construction date', 'fixed', 'discrete'], 
             'YearRemodAdd':['Remodel date (same as construction date if no remodeling or additions)', 'fixed', 'discrete'],
             'RoofStyle':['Type of roof', 'fixed', 'nominal'], 
             'RoofMatl':['Roof material', 'mutable', 'nominal'], 
             'Exterior1st':['Exterior covering on house', 'mutable', 'nominal'], 
             'Exterior2nd':['Exterior covering on house (if more than one material)', 'mutable', 'nominal'], 
             'MasVnrType':['Masonry veneer type', 'mutable', 'nominal'],
             'MasVnrArea':['Masonry veneer area in square feet', 'mutable', 'continuous'], 
             'ExterQual':['Evaluates the quality of the material on the exterior', 'mutable', 'nominal'], 
             'ExterCond':['Evaluates the present condition of the material on the exterior', 'mutable', 'nominal'], 
             'Foundation':['Type of foundation', 'fixed', 'nominal'], 
             'BsmtQual':['Evaluates the height of the basement', 'mutable', 'nominal'],
             'BsmtCond':['Evaluates the general condition of the basement', 'mutable', 'nominal'], 
             'BsmtExposure':['Refers to walkout or garden level walls', 'fixed', 'nominal'], 
             'BsmtFinType1':['Rating of basement finished area', 'mutable', 'nominal'], 
             'BsmtFinSF1':['Type 1 finished square feet', 'fixed', 'continuous'],
             'BsmtFinType2':['Rating of basement finished area (if multiple types)', 'mutable', 'nominal'], 
             'BsmtFinSF2':['Type 2 finished square feet', 'fixed', 'continuous'], 
             'BsmtUnfSF':['Unfinished square feet of basement area', 'fixed', 'continuous'], 
             'TotalBsmtSF':['Total square feet of basement area', 'fixed', 'continuous'], 
             'Heating':['Type of heating', 'mutable', 'nominal'],
             'HeatingQC':['Heating quality and condition', 'mutable', 'nominal'], 
             'CentralAir':['Central air conditioning', 'mutable', 'ordinal'], 
             'Electrical':['Electrical system', 'mutable', 'nominal'], 
             '1stFlrSF':['First Floor square feet','fixed', 'continuous'], 
             '2ndFlrSF':['Second floor square feet','fixed', 'continuous'],
             'LowQualFinSF':['Low quality finished square feet (all floors)', 'mutable', 'continuous'], 
             'GrLivArea':['Above grade (ground) living area square feet', 'fixed', 'continuous'], 
             'BsmtFullBath':['Basement full bathrooms', 'fixed', 'discrete'], 
             'BsmtHalfBath':['Basement half bathrooms', 'fixed', 'discrete'], 
             'FullBath':['Full bathrooms above grade', 'fixed', 'discrete'],
             'HalfBath':['Half baths above grade', 'fixed', 'discrete'], 
             'BedroomAbvGr':['Bedrooms above grade (does NOT include basement bedrooms)', 'fixed', 'discrete'], 
             'KitchenAbvGr':['Kitchens above grade', 'fixed', 'discrete'], 
             'KitchenQual':['Kitchen quality', 'mutable', 'nominal'],
             'TotRmsAbvGrd':['Total rooms above grade (does not include bathrooms)', 'fixed', 'discrete'], 
             'Functional':['Home functionality (Assume typical unless deductions are warranted)', 'mutable', 'nominal'], 
             'Fireplaces':['Number of fireplaces', 'fixed', 'discrete'], 
             'FireplaceQu':['Fireplace quality', 'mutable', 'nominal'], 
             'GarageType':['Garage location','fixed','nominal'],
             'GarageYrBlt':['Year garage was built','fixed','discrete'], 
             'GarageFinish':['Interior finish of the garage','mutable','nominal'], 
             'GarageCars':['Size of garage in car capacity','fixed','discrete'], 
             'GarageArea':['Size of garage in square feet','fixed','continuous'], 
             'GarageQual':['Garage quality','mutable','nominal'],
             'GarageCond':['Garage condition','mutable','nominal'], 
             'PavedDrive':['Paved driveway','mutable','nominal'], 
             'WoodDeckSF':['Wood deck area in square feet','fixed','continuous'], 
             'OpenPorchSF':['Open porch area in square feet','fixed','continuous'],
             'EnclosedPorch':['Enclosed porch area in square feet','fixed','continuous'], 
             '3SsnPorch':['Three season porch area in square feet','fixed','continuous'], 
             'ScreenPorch':['Screen porch area in square feet','fixed','continuous'], 
             'PoolArea':['Pool area in square feet','fixed','continuous'], 
             'PoolQC':['Pool quality','mutable','nominal'],
             'Fence':['Fence quality','mutable','nominal'], 
             'MiscFeature':['Miscellaneous feature not covered in other categories','fixed','nominal'], 
             'MiscVal':['$Value of miscellaneous feature','fixed','continuous'], 
             'MoSold':['Month Sold (MM)','fixed','discrete'], 
             'YrSold':['Year Sold (YYYY)','fixed','discrete'], 
             'SaleType':['Type of sale','fixed','nominal'],
             'SaleCondition':['Condition of sale','fixed','nominal'], 
             'SalePrice':['Value of property at sale','fixed','continuous-price']}

## Data Cleaning and Processing

In [ ]:
# display a summary for quantitative variables
house.describe()

In [ ]:
# display a summary for categorical variables
house.describe(include = 'object').sort_values(by = 'freq', ascending = False, axis = 1)

**Utilities** and **Street** only have 1 and 6 entries that have different values from the rest.
Their very low variability means they will not be good predictors, so these columns will be dropped.  

In [ ]:
house.drop(columns = ['Utilities','Street'], inplace = True)
del data_dict['Utilities']
del data_dict['Street']

**Condition1** and **Condition2** describe proximity to main roads, avenues, railways, parks, etc... using the same variables.  
Although some of them are clearly positive or negative, it is not possible to rank them given the available information. Since Condition2 shows a second condition only when present, there is very little variation between the two columns and they will be checked for repetition.  

In [ ]:
# display Condition1 values
house['Condition1'].value_counts()

In [ ]:
# display Condition2 values
house['Condition2'].value_counts()

In [ ]:
# check repetitions in the two columns
pd.set_option('display.max_rows', 50)
house[['Condition1', 'Condition2']][house['Condition1'] == house['Condition2']]

As expected, repetitions between the two columns occur when no particular conditions exist.  
Condition2 will be mapped to **None** when its value is equal to Condition1 and will be maintained.  


In [ ]:
# map Condition2 to None when is equal to Condition1
house.loc[house['Condition1'] == house['Condition2'], 'Condition2'] = 'None'

# display Condition2 updated values
house['Condition2'].value_counts()

**Exterior1st** and **Exterior2nd** describe the external finishes using the same variables.  
Since Exterior2nd shows a second material only when present, there is very little variation between the two columns and they will be checked for repetition.

In [ ]:
# display Exterior1st values
house['Exterior1st'].value_counts()

In [ ]:
# display Exterior2nd values
house['Exterior2nd'].value_counts()

In [ ]:
# create corrected spellings for the materials from comparing to data description
incorrect_sp = ['CmentBd', 'Wd Shng', 'Brk Cmn']
correct_sp = ['CemntBd', 'WdShing', 'BrkComm']

# replace spelling mistakes in Exterior2nd
house['Exterior2nd'] = house['Exterior2nd'].replace(incorrect_sp, correct_sp)

In [ ]:
# check repetitions in the two columns
house[['Exterior1st', 'Exterior2nd']][house['Exterior1st'] == house['Exterior2nd']]

In [ ]:
# map Exterior2nd to None when is equal to Exterior1st
house.loc[house['Exterior1st'] == house['Exterior2nd'], 'Exterior2nd'] = 'None' 

# display Exterior2nd updated values
house['Exterior2nd'].value_counts()

**MSZoning** describes the general zoning classification of the property.  
**RL**, **RM**, **FV** and **RH** indicate residential properties. Non-residential properties will be removed.  

In [ ]:
# check MSZoning column
house['MSZoning'].value_counts()

In [ ]:
# remove non-residential properties from the dataset
house = house[house['MSZoning'] != 'C (all)']

In [ ]:
house['MSZoning'].value_counts()

In [ ]:
house.shape

In [ ]:
# count the missing values for each column, display only the ones with missing values
na_count = house.isna().sum()
na_count[na_count != 0]

Several features have a lot of missing values (NaN).  
Their variable types will be checked first in order to decide how to process them.  

<br/><br/>

In [ ]:
# split variables into continuous, discrete, ordinal and nominal
continuous_vars = []
discrete_vars = []
ordinal_vars = []
nominal_vars = []

for k,v in data_dict.items():
    if v[2] == 'continuous':
        continuous_vars.append(k)
    elif v[2] == 'discrete':
        discrete_vars.append(k)
    elif v[2] == 'ordinal':
        ordinal_vars.append(k)
    elif v[2] == 'nominal':
        nominal_vars.append(k)

In [ ]:
# define quantitative and categorical variables
quant_vars = continuous_vars + discrete_vars
categ_vars = ordinal_vars + nominal_vars
price = ['SalePrice']

In [ ]:
# display variable types
print('CONTINUOUS VARIABLES \n', continuous_vars+price)
print()
print('DISCRETE VARIABLES \n', discrete_vars)
print()
print('ORDINAL VARIABLES \n', ordinal_vars)
print()
print('NOMINAL VARIABLES \n', nominal_vars)
print()

In general, rows can dropped for cases where data appears to be genuinely missing and rows account for less than 5% of data.  
The data description indicates that 'NA' means the property does not have a particular feature, rather than being a missing value.  
Categorical variables should have 'NA' values replaced with 'None' to avoid missing values, and all quantitative variables should be encoded as integers or floats.  
For **quantitative** variables missing values will be imputed using the most correlated features, while for **categorical** variables ordinal encoding (higher quality = higher value) will be performed where values are not correctly designated.  

<br/><br/>

In [ ]:
# drop rows with missing data
house.dropna(subset = ['MasVnrType', 'MasVnrArea','Electrical'], inplace = True)

In [ ]:
# convert NA to None - categorical variables where NA means none (e.g. alley = no alley)
house[['Alley','GarageType','MiscFeature']] = house[['Alley','GarageType','MiscFeature']].fillna('None')

In [ ]:
# check data types for quantitative variables
house[quant_vars].dtypes

In [ ]:
# find the most correlated variables with LotFrontage
for x, y in house.corr().items():
    if x == 'LotFrontage':
        df = pd.DataFrame({'variable':y.index, 'correlation': abs(y.values)})

df.sort_values(by = 'correlation', ascending = False).head()

In [ ]:
# impute the missing values for LotFrontage based on 1stFlrSF, lotArea and GrLivArea
imputer = KNNImputer(n_neighbors=2)
house[['LotFrontage','1stFlrSF','LotArea','GrLivArea']] = imputer.fit_transform(house[['LotFrontage','1stFlrSF',
                                                                                       'LotArea','GrLivArea']])

In [ ]:
# find the most correlated variables with GarageYrBlt
for x, y in house.corr().items():
    if x == 'GarageYrBlt':
        df = pd.DataFrame({'variable':y.index, 'correlation': abs(y.values)})

df.sort_values(by = 'correlation', ascending = False).head()

In [ ]:
# impute the missing values for GarageYrBlt based on YearBuilt, YearRemodAdd and GarageCars
imputer = KNNImputer(n_neighbors=2)
house[['GarageYrBlt','YearBuilt','YearRemodAdd','GarageCars']] = imputer.fit_transform(house[['GarageYrBlt','YearBuilt',
                                                                                              'YearRemodAdd','GarageCars']])

In [ ]:
# convert GarageYrBlt to integers
house = house.astype({'GarageYrBlt': int})

In [ ]:
# perform ordinal encoding - categorical variables where values are not correctly designated
# note: where the house does not have the feature ('NA') it will be assigned a value of 0

# create a list of qualitative variables (values = 'Ex', 'Gd', 'TA', 'Fa', 'PO', 'NA')
qual_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu',
             'GarageQual','GarageCond','PoolQC']

# for each column map each value with a new encoded value
for col in qual_vars:
    house[col] = house[col].map({'Ex'   : 5, 
                                 'Gd'   : 4, 
                                 'TA'   : 3, 
                                 'Fa'   : 2, 
                                 'Po'   : 1, 
                                 np.nan : 0})
    
# map LandSlope to encoded values
house['LandSlope'] = house['LandSlope'].map({'Gtl' : 3, 
                                             'Mod' : 2,
                                             'Sev' : 1})
    
# map BsmtExposure to encoded values 
house['BsmtExposure'] = house['BsmtExposure'].map({'Gd'   : 4, 
                                                   'Av'   : 3, 
                                                   'Mn'   : 2, 
                                                   'No'   : 1, 
                                                   np.nan : 0})

# map basement finishes to encoded values
finish_vars =['BsmtFinType1','BsmtFinType2']
for col in finish_vars:
    house[col] = house[col].map({'GLQ'  : 6, 
                                 'ALQ'  : 5, 
                                 'BLQ'  : 4, 
                                 'Rec'  : 3, 
                                 'LwQ'  : 2, 
                                 'Unf'  : 1, 
                                 np.nan : 0})

# map Functional to encoded values    
house['Functional'] = house['Functional'].map({'Typ'  : 8, 
                                               'Min1' : 7, 
                                               'Min2' : 6, 
                                               'Mod'  : 5, 
                                               'Maj1' : 4, 
                                               'Maj2' : 3,
                                               'Sev'  : 2,
                                               'Sal'  : 1})   
    
# map GarageFinish to encoded values
house['GarageFinish'] = house['GarageFinish'].map({'Fin'  : 3, 
                                                   'RFn'  : 2, 
                                                   'Unf'  : 1, 
                                                   np.nan : 0})

# map PavedDrive to encoded values
house['PavedDrive'] = house['PavedDrive'].map({'Y' : 3, 
                                               'P' : 2, 
                                               'N' : 1})

# map Fence to encoded values
house['Fence'] = house['Fence'].map({'GdPrv' : 4,
                                     'MnPrv' : 3,
                                     'GdWo'  : 2, 
                                     'MnWw'  : 1, 
                                     np.nan  : 0})

In [ ]:
# update data dictionary and variable lists
nominal_to_ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu',
                      'GarageQual','GarageCond','PoolQC', 'LandSlope', 'BsmtExposure', 'BsmtFinType1',
                      'BsmtFinType2', 'Functional', 'GarageFinish', 'PavedDrive', 'Fence']

for var in nominal_to_ordinal:
    data_dict[var][2] = 'ordinal'
    ordinal_vars.append(var)
    nominal_vars.remove(var)

In [ ]:
# check the shape
house.shape

In [ ]:
# count the missing values for each column
house.isna().sum()

In [ ]:
# create a list of variables where 0 refers to a non existent feature (e.g. a basement)
zero_features = ['MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath',
                 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageArea', 'WoodDeckSF',
                 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'Fence', 'MiscVal']

# determine the proportion of houses that do not have that feature
zero_count = [(house[x] == 0).sum() for x in zero_features]
zero_proportion = [((house[x] == 0).sum()) / ((house[x].notna()).sum()) for x in zero_features]

# create and display a dataframe with the proportions
zeros = pd.DataFrame({'feature' : zero_features, 'zero_count' : zero_count, 'zero_proportion' : zero_proportion})
zeros.sort_values(by = 'zero_proportion', ascending = False)

**PoolArea** has a value greater than 0 in 7 entries, meaning that only 7 properties have a swimming pool.  
This feature is unlikely to be a good predictor, however it will be maintained to understand its impact.  

In [ ]:
# create a new variable for total square feet above ground
house['TotalSFAbvGrd'] = house['1stFlrSF'] + house['2ndFlrSF']

# create a new variable for total square feet
house['TotalSF'] = house['TotalBsmtSF'] + house['1stFlrSF'] + house['2ndFlrSF']

# create a new variable for lot open area
house['LotOpenArea'] = house['LotArea'] - (house['1stFlrSF'] + house['OpenPorchSF'] + house['EnclosedPorch'] +
                                           house['3SsnPorch'] + house['ScreenPorch'])

# create a new variable for floor area ratio
house['FloorAreaRatio'] = (house['1stFlrSF'] + house['2ndFlrSF']) / house['LotArea']

# create a new variable for total bathrooms
house['TotalBath'] = house['FullBath'] + house['BsmtFullBath'] + 0.5*house['HalfBath'] + 0.5*house['BsmtHalfBath']

# create a new variable for property age (at sale)
house['Age'] = house['YrSold'] - house['YearBuilt']

In [ ]:
# update data dictionary and variable lists
data_dict['TotalSFAbvGrd'] = ['Total square feet above ground', 'fixed', 'continuous']
data_dict['TotalSF'] = ['Total square feet (all floors)', 'fixed', 'continuous']
data_dict['LotOpenArea'] = ['Lot area vertically uncovered by any structure, in square feet', 'fixed', 'continuous']
data_dict['FloorAreaRatio'] = ['Ratio of the total floor area (above ground) to the lot size', 'fixed', 'continuous']
data_dict['TotalBath'] = ['Total number of bathrooms (all floors)', 'fixed', 'discrete']
data_dict['AgeAtSale'] = ['Age of the property at time of sale', 'fixed', 'discrete']

new_continuous = ['TotalSFAbvGrd', 'TotalSF', 'LotOpenArea', 'FloorAreaRatio']
new_discrete = ['TotalBath', 'AgeAtSale']

for var in new_continuous:
    continuous_vars.append(var)
    quant_vars.append(var)

for var in new_discrete:
    discrete_vars.append(var)
    quant_vars.append(var)

In [ ]:
# split variables into fixed and mutable
fixed_vars = []
mutable_vars = []

for k,v in data_dict.items():
    if v[1] == 'fixed':
        fixed_vars.append(k)
    elif v[1] == 'mutable':
        mutable_vars.append(k)

In [ ]:
# display variable types
print('FIXED VARIABLES \n', fixed_vars)
print()
print('MUTABLE VARIABLES \n', mutable_vars)
print()

In [ ]:
# display variable types
print('CONTINUOUS VARIABLES \n', continuous_vars+price)
print()
print('DISCRETE VARIABLES \n', discrete_vars)
print()
print('ORDINAL VARIABLES \n', ordinal_vars)
print()
print('NOMINAL VARIABLES \n', nominal_vars)
print()

In [ ]:
# display a summary for quantitative variables
house.describe()

In [ ]:
# display a summary for categorical variables
house.describe(include = 'object').sort_values(by = 'freq', ascending = False, axis = 1)

## Exploratory Data Analysis

In [ ]:
# explore correlations between continuous variables and price

plt.figure(figsize=(15, 15))
sns.heatmap(house[continuous_vars+price].corr(), cmap='Blues', annot=True)
plt.show;

In [ ]:
# explore correlations between discrete variables, ordinal variables and price

plt.figure(figsize=(15, 15))
sns.heatmap(house[discrete_vars+ordinal_vars+price].corr(), cmap='Blues', annot=True)
plt.show;

In [ ]:
cont_vars_sub = ['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']

disc_vars_sub = ['FullBath', 'TotRmsAbvGrd', 'GarageCars', 'OverallQual']

In [ ]:
sns.set_theme(style='white' , font_scale=1.5)

plt.figure(figsize=(15, 15))

sns.pairplot(house[cont_vars_sub+price], kind='reg', height=3.5, plot_kws=dict(marker="o"),diag_kws=dict(fill=True),
             corner=True);


In [ ]:
sns.set_theme(style='white' , font_scale=1.5)

plt.figure(figsize=(15, 15))

sns.pairplot(house[disc_vars_sub+price], kind='reg', height=3.5, plot_kws=dict(marker="o"),diag_kws=dict(fill=True),
             corner=True);


In [ ]:
house.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
null_data = house[house.isnull().any(axis=1)]

In [ ]:
null_data

In [ ]:
# Identify fixed features that can predict price.

"""

For the fixed features I have not only considered the ones that cannot be renovated, but also the ones that are unlikely
to be renovated by the real estate company because of the huge cost involved.


LotFrontage
LotArea
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
BsmtQual
TotalBsmtSF
1stFlrSF
2ndFlrSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageType
GarageArea

"""



In [ ]:
# copy the original dataframe.
# the new dataframe will be used for initial modelling

df = house.copy().reset_index()
df

In [ ]:
fixed_features = ['LotFrontage', 'LotArea', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
                  'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'BsmtQual', 'TotalBsmtSF',
                  '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
                  'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageType', 'GarageArea']

fixed_cat_vars = [x for x in df.columns if x in fixed_features and x in categ_vars]


In [ ]:
fixed_cat_vars

In [ ]:
other_features = [x for x in df.columns if x not in fixed_features and x != 'SalePrice']

In [ ]:
other_features

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
# dummify the categorical variables

df_dum = pd.get_dummies(df, columns=fixed_cat_vars, drop_first=True)


In [ ]:
df_dum.head()

In [ ]:
# define target variable

y = df_dum.pop('SalePrice')

In [ ]:
# define predictor matrix

X = df_dum.drop(columns=other_features)

In [ ]:
X.head()

In [ ]:
df_dum.YrSold.value_counts()

In [ ]:
# create train and test sets based on YrSold

X_train = X[df_dum.YrSold < 2010]
X_test = X[df_dum.YrSold == 2010]

y_train = y[df_dum.YrSold < 2010]
y_test = y[df_dum.YrSold == 2010]

In [ ]:
# train and test set baseline

print(y_train.value_counts(normalize=True).max())
print(y_test.value_counts(normalize=True).max())

In [ ]:
# rescale the dataset

scaler = StandardScaler()

In [ ]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# instatiate the model
model = LinearRegression()

# fit the model
model.fit(X_train, y_train)

# score the model
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
print('Intercept:', model.intercept_)

lr_coeffs = pd.DataFrame(list(zip(X_train.columns, model.coef_)))
lr_coeffs

In [ ]:
# best estimate of price

lr_coeffs[lr_coeffs[1]==lr_coeffs[1].max()]

In [ ]:
# best estimates of price

lr_coeffs.nlargest(5,[1])

In [ ]:
# worst estimates of price

lr_coeffs.nsmallest(5,[1])

In [ ]:
# perform 5-fold cross validation

scores = cross_val_score(model, X, y, cv=5)
print("Cross-validated scores:", scores)
print("Mean of Ccoss-validated scores:", scores.mean())

In [ ]:
# A:

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovatable features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [ ]:
"""

I am going to use the changeable features in the predictor matrix and fit a new model.


"""

In [ ]:
# copy the updated dataframe to create a new predictor matrix

df2 = df.copy().reset_index()
df2

In [ ]:
df2.drop(columns=['level_0', 'index'], inplace=True)
df2

In [ ]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. It is recommended to do some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [ ]:
# A: